In [1]:
import pyaudio
import numpy as np
import wave
import keyboard
import time

#set 's' as the start key 
Start_key='s'

background=0

#record the background
def record_background():
    CHUNK = 1600
    resolution = pyaudio.paInt16 
    CHANNELS = 1 
    sampling_rate = 16000
    record_time= 1
    Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    #time.sleep(0.2)
    data = stream.read(10)
    np_array=np.array(data)
        
    #convert hexadecimal numbers to decimal ones
    int_np_array=np.frombuffer(np_array, np.int16)     
    int_64 = np.array(int_np_array,dtype='int64')
    energy=[0 for j in range(len(int_64))]
    for i in range(len(int_64)):
        if int_64[i]==0:
            e=10*np.log10((np.square(int_64[i]+(1e-3))))
            energy[i]=e
        else:
            e=10*np.log10((np.square(int_64[i])))
            energy[i]=e
    background=np.mean(energy)

    #stop recording
    stream.stop_stream()
    stream.close()
    p.terminate()
    print(background)
    return background

def record(background):
    #initialize parameters for recording with pyaudio
    CHUNK = 1600
    resolution = pyaudio.paInt16
    CHANNELS = 1
    sampling_rate = 16000
    
    #initialize parameters for classifying frames
    forgetfactor = 1
    isSpeech = False
    level = 0
    adjustment = 0.05
    threshold = 8
    init = 0
    count = 0
    frame_num = 0
    
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    frames=[]
    intframes=[]
    while True:
        #get data
        #time.sleep(0.2)
        data = stream.read(10)
        frames.append(data)
        np_array=np.array(data)
        
        #convert hexadecimal numbers to decimal ones
        int_np_array=np.frombuffer(np_array, np.int16)
        intframes+=int_np_array.tolist()
        int_64 = np.array(int_np_array,dtype='int64')
        energy=[0 for j in range(len(int_64))]
        for i in range(len(int_64)):
            if int_64[i]==0:
                e=10*np.log10((np.square(int_64[i]+(1e-3))))
                energy[i]=e
            else:
                e=10*np.log10((np.square(int_64[i])))
                energy[i]=e
        
        #initialize background and level
        if init==0:
            init+=1
            level=energy[0]
        energy=np.mean(energy)

        #adaptive endpointing algo
        current=energy
        level=((level*forgetfactor)+current)/(1+forgetfactor)
        if(current<background):
            background=current
        else:
            background+=(current-background)*adjustment
        if(level<background): level=background
        if(level-background>threshold):
            count=0
            isSpeech = True
        else: 
            count+=1
        if count>40: isSpeech=False
        print((level,background,level-background))
        
        #recording end condition
        if not isSpeech and init==1 and count>40:
            print("stop recording")
            stream.stop_stream()
            stream.close()
            p.terminate()
            break
    return intframes

def listen():
    intframes=[]
    while True:
        if keyboard.is_pressed(Start_key):
            print('start')
            background=record_background()
            intframes=record(background)
            break
    return intframes

In [2]:
intframes=listen()

start
-48.0
(-48.0, -48.0, 0.0)
(-54.0, -60.0, 6.0)
(-54.0, -59.7, 5.700000000000003)
(-51.0, -59.115, 8.115000000000002)
(-55.5, -60.0, 4.5)
(-54.75, -59.7, 4.950000000000003)
(-54.375, -59.415000000000006, 5.040000000000006)
(-54.1875, -59.14425000000001, 4.956750000000007)
(-42.09375, -57.68703750000001, 15.59328750000001)
(-45.046875, -57.20268562500001, 12.155810625000008)
(-49.5234375, -57.042551343750006, 7.519113843750006)
(-45.76171875, -56.290423776562506, 10.528705026562506)
(-49.880859375, -56.17590258773438, 6.295043212734377)
(-48.9404296875, -55.76710745834766, 6.82667777084766)
(-42.47021484375, -54.77875208543028, 12.308537241680277)
(-45.235107421875, -54.43981448115876, 9.204707059283763)
(-46.6175537109375, -54.11782375710082, 7.5002700461633225)
(-50.30877685546875, -54.11193256924578, 3.8031557137770307)
(-46.154388427734375, -53.506335940783494, 7.351947513049119)
(-50.07719421386719, -54.0, 3.9228057861328125)
(-49.038597106933594, -53.7, 4.661402893066409)
(-39

In [3]:
print(intframes)
#print(10*np.log(np.sum(np.square(intframes[-1]))))

[0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, -1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, -1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, -1, 1, 0, 0, 0, -1, -1, 0, 0, 0, -1, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, -1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0,

In [4]:
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)